# 패키지

In [127]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

# 데이터

In [128]:
path1 = 'C:/Users/samsung/Desktop/수업자료/train.csv'
path2 = 'C:/Users/samsung/Desktop/수업자료/test.csv'
path3 = 'C:/Users/samsung/Desktop/수업자료/sample_submission.csv'


train = pd.read_csv(path1)
test = pd.read_csv(path2)
submission = pd.read_csv(path3)

In [129]:
train.shape, test.shape, submission.shape

((2952, 15), (1022, 14), (150, 2))

In [130]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


# 데이터 오류 처리(총세대수 합계 오류)

In [131]:
small_diff = ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']
big_diff = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804']
diff = small_diff + big_diff

for i in diff:
    train_target = train[train['단지코드']==i].copy()
    train_change = train[train['단지코드']==i].전용면적별세대수.sum()
    k = len(train_target)
    
    if k != 0:
        train_target['총세대수'] = train_change
        
        train[train['단지코드']==i] = train_target
        
for j in diff:
    test_target = test[test['단지코드']==j].copy()
    test_change = test[test['단지코드']==j].전용면적별세대수.sum()
    k = len(test_target)
    
    if k != 0:
        test_target['총세대수'] = test_change
        
        test[test['단지코드']==j] = test_target
    

In [132]:
result_list = []

for i in diff:
    train_total = train[train['단지코드']==i]['총세대수']
    train_sub = train[train['단지코드']==i].전용면적별세대수.sum()
    k = len(train_total)
    
    if k != 0:
        if train_total.iloc[0] == train_sub:
            result_list.append('ok')
        else:
            result_list.append('fail')
            
for j in diff:
    test_total = test[test['단지코드']==j]['총세대수']
    test_sub = test[test['단지코드']==j].전용면적별세대수.sum()
    k = len(test_total)
    
    if k != 0:
        if test_total.iloc[0] == test_sub:
            result_list.append('ok')
        else:
            result_list.append('fail')
            
for i in result_list:
    if i == 'ok':
        continue
    print('오류 제거 실패')
    
print('완료')

완료


# 데이터 오류 처리(트레인셋에서 제거)

In [133]:
overlap_list = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036']
mistake_list = ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']

err_list = overlap_list + mistake_list

for i in err_list:
    idx = train[train['단지코드']==i].index
    train.drop(idx,inplace= True)

In [134]:
chk_num = 0

for i in err_list:
    k = len(train[train['단지코드']==i])
    if k == 0:
        chk_num += 1
        
if chk_num == len(err_list):
    print('오류 제거 완료')

오류 제거 완료


# 결측치 확인

In [135]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    207
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [136]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

# 결측치 처리(임대보증금, 임대료)

In [145]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

In [146]:
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

# 결측치 처리(지하철 역 수)

In [155]:
subway_null_codes = train[train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
train.loc[train.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)

Series([], Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64)

In [156]:
subway_null_codes = test[test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
test.loc[test.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)

Series([], Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64)

In [154]:
cols = '도보 10분거리 내 지하철역 수(환승노선 수 반영)'
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

# 결측치 처리(자격유형)

In [159]:
test[test.단지코드=='C2411']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
193,C2411,961,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
194,C2411,961,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
195,C2411,961,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
196,C2411,961,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
197,C2411,961,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000.0,171480.0,0.0,2.0,840.0


In [160]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

In [161]:
test[test.단지코드=='C2253'].head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,0.0,0.0,0.0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,0.0,0.0,0.0,2.0,173.0


In [162]:
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

# 중복 제거

In [165]:
train = train.drop_duplicates()
test = test.drop_duplicates()

# 컬럼명 변경

In [166]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [167]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적',
    '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형',
    '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

# 지역명 숫자로 매핑(제거)

In [168]:
# 한 컬럼에 각기 다른 숫자형 데이터로 변경한 후,
# 이 데이터 값을 머신러닝 알고리즘에 그대로 넣어 데이터를 예측 하라고 지시한다면
# 컴퓨터가 이들 값의 관계를 예상과 다르게 형성할 가능성이 있다

In [169]:
# local_map = {}

# for i,loc in enumerate(train['지역'].unique()):
#     local_map[loc] = i

# local_map

In [170]:
# train['지역'] = train['지역'].map(local_map)
# test['지역'] = test['지역'].map(local_map)

# 원핫인코딩을 통한 지역명 변경

In [171]:
train = pd.get_dummies(data = train, columns = ['지역'], prefix = '지역')
test = pd.get_dummies(data = test, columns = ['지역'], prefix = '지역')

# 전용면적을 5의 배수로 변경

In [172]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

# 전용면적 상/하한 적용

In [173]:
# 상한100, 하한 15

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [174]:
train['전용면적'].unique()

array([ 30.,  35.,  45.,  50.,  40.,  55.,  25.,  70.,  15.,  20., 100.,
        60.,  75.,  80.,  65.])

# 단지별 데이터 1차원으로 취합

In [175]:
seoul = train.지역_서울특별시.copy()
seoul

8       0
9       0
10      0
11      0
12      0
       ..
2947    0
2948    0
2949    0
2950    0
2951    0
Name: 지역_서울특별시, Length: 2549, dtype: uint8

In [176]:
del train['지역_서울특별시']

In [177]:
train['지역_서울특별시'] = seoul

In [178]:
test['지역_서울특별시'] = 0

In [179]:
test

,단지코드,총세대수,임대건물구분,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,...,지역_대전광역시,지역_부산광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도,지역_서울특별시
0,C1072,754,아파트,국민임대,35.0,116,14.0,H,22830000.0,189840.0,...,0,0,0,0,0,0,0,0,0,0
1,C1072,754,아파트,국민임대,45.0,30,14.0,A,36048000.0,249930.0,...,0,0,0,0,0,0,0,0,0,0
2,C1072,754,아파트,국민임대,45.0,112,14.0,H,36048000.0,249930.0,...,0,0,0,0,0,0,0,0,0,0
3,C1072,754,아파트,국민임대,45.0,120,14.0,H,36048000.0,249930.0,...,0,0,0,0,0,0,0,0,0,0
4,C1072,754,아파트,국민임대,50.0,60,14.0,H,43497000.0,296780.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,670,아파트,행복주택,35.0,126,38.0,L,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1018,C2189,378,아파트,국민임대,25.0,96,45.0,H,6872000.0,106400.0,...,0,0,0,0,0,1,0,0,0,0
1019,C2189,378,아파트,국민임대,25.0,20,45.0,H,6872000.0,106400.0,...,0,0,0,0,0,1,0,0,0,0
1020,C2189,378,아파트,국민임대,35.0,202,45.0,H,13410000.0,144600.0,...,0,0,0,0,0,1,0,0,0,0


In [180]:
location = train.iloc[:,14:].columns.tolist()
location

['지역_강원도',
 '지역_경기도',
 '지역_경상남도',
 '지역_경상북도',
 '지역_광주광역시',
 '지역_대구광역시',
 '지역_대전광역시',
 '지역_부산광역시',
 '지역_세종특별자치시',
 '지역_울산광역시',
 '지역_전라남도',
 '지역_전라북도',
 '지역_제주특별자치도',
 '지역_충청남도',
 '지역_충청북도',
 '지역_서울특별시']

In [181]:
columns = ['단지코드', '총세대수', '공가수'] + location + [ '단지내주차면수','지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [182]:
columns

['단지코드',
 '총세대수',
 '공가수',
 '지역_강원도',
 '지역_경기도',
 '지역_경상남도',
 '지역_경상북도',
 '지역_광주광역시',
 '지역_대구광역시',
 '지역_대전광역시',
 '지역_부산광역시',
 '지역_세종특별자치시',
 '지역_울산광역시',
 '지역_전라남도',
 '지역_전라북도',
 '지역_제주특별자치도',
 '지역_충청남도',
 '지역_충청북도',
 '지역_서울특별시',
 '단지내주차면수',
 '지하철',
 '버스']

In [183]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [184]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

411it [00:21, 19.27it/s]
150it [00:07, 20.54it/s]


In [185]:
new_test

,단지코드,총세대수,공가수,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,...,면적_55.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0
0,C1072,754.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1128,1354.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C1456,619.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C1840,593.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C1332,1297.0,11.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,346.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,C1266,591.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,C2152,120.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0
148,C1267,670.0,38.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,74.0,0.0,50.0,240.0,0.0,0.0,0.0,0.0,0.0


# 결측치 처리

In [186]:
new_train.isna().sum().to_frame()

,0
단지코드,0
총세대수,0
공가수,0
지역_강원도,0
지역_경기도,0
지역_경상남도,0
지역_경상북도,0
지역_광주광역시,0
지역_대구광역시,0
지역_대전광역시,0


In [187]:
new_test.isna().sum().to_frame()

,0
단지코드,0
총세대수,0
공가수,0
지역_강원도,0
지역_경기도,0
지역_경상남도,0
지역_경상북도,0
지역_광주광역시,0
지역_대구광역시,0
지역_대전광역시,0


# 학습

In [188]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [189]:
y_train

0       205.0
1      1064.0
2       730.0
3       553.0
4       415.0
        ...  
406      57.0
407     246.0
408      19.0
409      16.0
410     146.0
Name: 등록차량수, Length: 411, dtype: float64

In [190]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [191]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

# 추론 및 제출

In [192]:
pred = model.predict(x_test)

In [193]:
submission['num'] = pred

In [194]:
submission

,code,num
0,C1072,723.50
1,C1128,1242.52
2,C1456,486.64
3,C1840,545.85
4,C1332,1072.30
...,...,...
145,C2456,275.15
146,C1266,489.46
147,C2152,32.80
148,C1267,429.75


In [195]:
submission.to_csv('preparing2.csv',index=False)